<a href="https://colab.research.google.com/github/Tshepo-Matlhabe/Portfolio/blob/main/Python_Football_Webscrapping_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Football Webscraping Project for the big 5 European Leagues (EPL, La Liga, Serie A, Bundesliga and Ligue 1)

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats'

In [ ]:
response = requests.get(url)
response

<Response [200]>

In [ ]:
soup = BeautifulSoup(response.text, 'html')

In [ ]:
#Use a css selector to select the table by tag and class name
standings_table = soup.select('table.stats_table')[0]

In [ ]:
# Using find_all to find 'tags' with href
team_links = standings_table.find_all('a')

In [ ]:
#Going through all the "a" elements to find the value of the href properties
links = [l.get("href") for l in team_links]

In [ ]:
#Getting rid of all links that do not contain 'squads' as I am only looking to get links that take me to the stats for the different premier league clubs
links = [l for l in links if '/squads/' in l]

In [ ]:
# Turning the links into 'active urls' using format string
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

['https://fbref.com/en/squads/c7a9f859/Bayer-Leverkusen-Stats',
 'https://fbref.com/en/squads/d609edc0/Internazionale-Stats',
 'https://fbref.com/en/squads/53a2f082/Real-Madrid-Stats',
 'https://fbref.com/en/squads/054efa67/Bayern-Munich-Stats',
 'https://fbref.com/en/squads/e2d8892c/Paris-Saint-Germain-Stats',
 'https://fbref.com/en/squads/9024a00a/Girona-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/e0652b02/Juventus-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/dc56fe14/Milan-Stats',
 'https://fbref.com/en/squads/206d90db/Barcelona-Stats',
 'https://fbref.com/en/squads/598bc722/Stuttgart-Stats',
 'https://fbref.com/en/squads/db3b9613/Atletico-Madrid-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/2b390eca/Athletic-Club-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-V

##Extracting Match stats using Pandas and Requests

In [ ]:
# Getting match stats for individual teams
team_url = team_urls[0]

In [ ]:
response2 = requests.get(team_url)

In [ ]:
#Using pandas and match to search for the 'scores and fixtures' string in the html and get that table in a dataframe

import pandas as pd

matches = pd.read_html(response2.text, match = 'Scores & Fixtures')[0]

In [ ]:
#Initializing a list that will contain dataframes containing matchlogs for each team for each season
all_matches = []

In [ ]:
#Same url I used Initially
standings_url = 'https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats'

In [ ]:
import requests
from bs4 import BeautifulSoup

# Selecting the range of years
years = list(range(2024, 2019, -1))

# Create lists to store links for each year
links_2024 = []
links_2023 = []
links_2022 = []
links_2021 = []

for year in years:
    print(f'processing year: {year}')


    response = requests.get(standings_url)
    soup = BeautifulSoup(response.text, 'html')

    try:
        # Use a CSS selector to select the table by tag and class name
        standings_table = soup.select('table.stats_table')[0]
    except IndexError as e:
        print(f"Could not find standings table in {year}: {e}")
        continue

    # Find all 'a' elements within the table
    team_links = standings_table.find_all('a')

    # Extract href properties from 'a' elements
    links = [l.get("href") for l in team_links]

    # Filter the links to only get the links to squads
    links = [l for l in links if '/squads/' in l]

    # Convert the links to absolute URLs
    team_urls = [f"https://fbref.com{l}" for l in links]

    # Append the team URLs to the list corresponding to the current year
    if year == 2024:
        links_2024.extend(team_urls)
    elif year == 2023:
        links_2023.extend(team_urls)
    elif year == 2022:
        links_2022.extend(team_urls)
    elif year == 2021:
        links_2021.extend(team_urls)

    # Get the URL for the previous season
    previous_season = soup.select('a.prev')[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"

# Print the different lists with the links for their respective years
print("Links for year 2024:")
for link in links_2024:
    print(link)

print("\nLinks for year 2023:")
for link in links_2023:
    print(link)

print("\nLinks for year 2022:")
for link in links_2022:
    print(link)

print("\nLinks for year 2021:")
for link in links_2021:
    print(link)


In [ ]:
all_matches3= []

In [ ]:
team_urls = links_2023

In [ ]:

import time
#2. Looping through ech team urls to get their individual match logs
for team_url in team_urls:
  #Splitting the team link with the last forward slash as that contains the portion of the link where the team name resides
  team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")

  response2 = requests.get(team_url)

  try:
      # Using pandas and match to search for the 'scores and fixtures' string in the html and get that table in a dataframe
      matches = pd.read_html(response2.text, match='Scores & Fixtures')
  except ValueError as e:
      print(f"Error processing Scores and fixtures table for {team_name} {e}")
      continue

  # Finding the url to the shooting data from the team page
  soup2 = BeautifulSoup(response2.text)

  # Finding all a tags in the page
  links = soup2.find_all('a')

  # Using a list comprehension to get the actual url of the link
  links = [l.get('href') for l in links]

  # Filtering with only the shooting links
  links = [l for l in links if l and 'all_comps/shooting/' in l]

  # Convert the link to an absolute url
  data = requests.get(f"https://fbref.com{links[0]}")

  # Read in the shooting data into pandas
  shooting = pd.read_html(data.text, match='Shooting')[0]

  # Dropping the first heading as it is not useful and I don't want multi-level indexing
  shooting.columns = shooting.columns.droplevel()

  # Merging the matches dataframe with the shooting dataframe...
  # Some teams do not have shooting stats so I have exception handling for such cases
  try:
      if "Date" not in matches[0].columns or "Date" not in shooting.columns:
          raise KeyError("Date column not found in matches or shooting DataFrame")

      # Check if the date exists in both matches and shooting tables
      common_dates = set(matches[0]["Date"]).intersection(shooting["Date"])
      if not common_dates:
          print(f"No common dates found for {team_name} ")
          continue

      # Filter shooting stats table to include only common dates
      shooting_filtered = shooting[shooting["Date"].isin(common_dates)]

      # Merge the data
      team_data = matches[0].merge(shooting_filtered[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
  except ValueError as e:
      print(f"could not find shooting stats for {team_name} : {e}")
  except KeyError as k:
      print(f"no column match for {team_name} : {k}")
      continue

  # Adding the Seasons and Team columns to identify the season and the team name for the scores and fixtures data collected.
  # team_data['Season'] = year
  team_data['Team'] = team_name

  # Append the all_matches list with a list of these collected dataframes
  all_matches3.append(team_data)
  time.sleep(2)





In [ ]:
all_matches3

In [ ]:
#Combining all the individual
match_df2 = pd.concat(all_matches3)

In [ ]:
#Turning all columns to lowercase to make it easier to work with the data.
match_df2.columns = [c.lower() for c in match_df2.columns]

In [ ]:
match_df2

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,fk,pk,pkatt,team
0,2022-08-15,18:30,Serie A,Matchweek 1,Mon,Away,W,5,2,Hellas Verona,...,Michael Fabbri,Match Report,NaN,25.0,8.0,15.2,0.0,0,0,Napoli
1,2022-08-21,18:30,Serie A,Matchweek 2,Sun,Home,W,4,0,Monza,...,Francesco Fourneau,Match Report,NaN,22.0,5.0,15.3,1.0,0,0,Napoli
2,2022-08-28,20:45,Serie A,Matchweek 3,Sun,Away,D,0,0,Fiorentina,...,Livio Marinelli,Match Report,NaN,13.0,2.0,14.7,1.0,0,0,Napoli
3,2022-08-31,20:45,Serie A,Matchweek 4,Wed,Home,D,1,1,Lecce,...,Matteo Marcenaro,Match Report,NaN,19.0,7.0,17.7,0.0,0,0,Napoli
4,2022-09-03,20:45,Serie A,Matchweek 5,Sat,Away,W,2,1,Lazio,...,Simone Sozza,Match Report,NaN,19.0,7.0,16.1,0.0,0,0,Napoli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2023-05-07,13:00,Ligue 1,Matchweek 34,Sun,Home,L,1,2,Monaco,...,Jeremy Stinat,Match Report,NaN,10.0,3.0,19.8,1.0,0,0,Angers
37,2023-05-14,20:45,Ligue 1,Matchweek 35,Sun,Away,L,1,3,Marseille,...,Benoît Millot,Match Report,NaN,4.0,1.0,19.4,0.0,0,0,Angers
38,2023-05-21,15:00,Ligue 1,Matchweek 36,Sun,Away,D,2,2,Reims,...,Florent Batta,Match Report,NaN,7.0,3.0,14.1,0.0,0,0,Angers
39,2023-05-27,21:00,Ligue 1,Matchweek 37,Sat,Home,W,2,1,Troyes,...,Karim Abed,Match Report,NaN,18.0,5.0,20.0,1.0,0,0,Angers


In [ ]:
#Saving the df to csv
match_df2.to_excel('Matches2023-2022_season.xlsx', index=False)